# Komparatory

Komparatory to kolejny dobry przykład na zastosowanie polimorfizmu w Javie. W sytuacji gdy musimy posortować kolekcję liczb wydaje się to całkiem naturalne i nie myślimy o tym jak to działa - wiemy, że liczby będą na koniec uporządkowane rosnąco lub malejąco. W przypadku bardziej złożonych struktur nie jest to już takie oczywiste. Skąd właściwie Java wie jak posortować napisy? I dlaczego nie wie jak posortować obiekty typu `Vector2d` albo nawet całe zwierzęta?

Warto zauważyć, że algorytm sortowania (nieważne jaki) będzie działał tak samo dla dowolnych danych, z wyjątkiem kluczowej operacji - porównania elementów. Trzeba więc jasno określić, który z dwóch obiektów jest większy/mniejszy, a więc wprowadzić tzw. relację porządku między obiektami danego typu. w Javie możemy do tego ponownie użyć interfejsów, i to na dwa sposoby.

## `Comparable`

Jeśli chcemy by obiekty danego typu realizowały relację porządku, wystarczy by implementowały interfejs `Comparable<T>`. Interfejs ten wymaga dorzucenia jednej metody, `compareTo()`:

```java
public class Vector2d implements Comparable<Vector2d> {
    
 ...
     @Override
     public int compareTo(Vector2d other) {
         return Integer.compare(x(), other.x());
     }
}
```
Rezultatem porównania może być liczba dodania (wtedy nasz obiekt jest większy), ujemna (mniejszy) lub 0 (równy). Algorytm sortowania może dzięki temu przyjąć bezpiecznie kolekcję elementów typu `Comparable<T>` nie wnikając w inne szczegóły na temat tego, co sortuje.

## `Comparator`

Co jeśli jednak czasem chcielibyśmy sortować nasze elementy w inny sposób, porównując je po innym kluczu. Przykładowo gdybyśmy sortowali kolory, mogliśmy czasem ustawiać je według jasności, a innym razem po odcieniu. W takim przypadku interfejs `Comparable` by nas ograniczał, bo na sztywno ustala taki a nie inny sposób sortowania i musielibyśmy albo zmieniać za każdym razem kod, albo np. mieć dwie różne klasy reprezentujące kolor. Nie jest to ani ładne, ani wygodne, dlatego lepiej w ogóle odseparować sposób sortowania od danych, które sortujemy:

```java
public class XComparator implements Comparator<Vector2d> {
    @Override
    public int compare(Vector2d vector1, Vector2d vector2) {
        return Integer.compare(vector1.x(), vector2.x());
    }
}
```
Teraz możemy takiego komparatora używać do sortowania w zupełnie niezależny sposób:
```java
Collections.sort(positions, new XComparator());
Collections.sort(positions, new YComparator());
```

## `TreeSet`

Możemy sortować kolekcje na żądanie, ale czasem chcemy utrzymywać porządek przez cały czas istnienia danej kolekcji elementów, również po jej rozszerzeniu o nowe elementy. Sortowanie na nowo po każdym dodaniu elementu to niekoniecznie dobry pomysł, bo taka operacja kosztuje czas (dokładniej: O(N\*log(N)) dla N elementów). Możemy tu skorzystać z `TreeSet`, którego implementacja opiera się o drzewa porządkujące na bieżąco dane. Tego typu kolekcja musi oczywiście wiedzieć jak porównywać elementy:

```java
SortedSet<Vector2d> xSet = new TreeSet<>(new XComparator());
```
Potem można już normalnie korzystać z niej jak z każdej innej kolekcji. Za każdym razem gdy wykonamy np. `xSet.add(vector)` nasz wektor ustawi się na właściwą pozycję w zbiorze (drzewie). Koszt takiego dodania to O(log(N)), czyli więcej niż np. dodanie do listy czy HashSetu, ale wciąż stosunkowo mało. 

## Sposoby porównywania

Porównywanie elementów może być właściwie dowolnym kawałkiem kodu. W przykladzie z pozycjami sprowadza się to tak naprwadę do:
```java
return vector1.x() - vector2.x();
```
Trzeba jednak bardzo uważać. Jeśli nasze koordynaty byłyby wyrażone jako **float**, bardzo łatwo moglibyśmy popełnić taki klasyczny błąd:
```java
return (int)(vector1.x() - vector2.x()); // co jeśli oba x są z przedziału <0, 1)?
```
W efekcie taka funkcja zwracałaby 0 dla zupełnie różnych wektorów, twierdząc że są one równe! Dlatego lepiej korzystać z wysokopoziomowych funkcji:

```java
return Float.compare(vector1.x(), vector2.x());

```

### Operatory funkcyjne

Jako że `Comparator` to interfejs z pojedynczą metodą, same komparatory możemy też zapisywać krócej, jako lambdy:

```java
SortedSet<Vector2d> xSet = new TreeSet<>((vector1, vector2) -> Integer.compare(vector1.x(), vector2.x()));
```

Albo jeszcze prościej, korzystając z nieco nowszych narzędzi w Javie:

```java
SortedSet<Vector2d> xSet = new TreeSet<>(Comparator.comparingInt(vector -> vector.x()));
```
albo jeszcze krócej:

```java
SortedSet<Vector2d> xSet = new TreeSet<>(Comparator.comparingInt(Vector2d::x));
```

W ten sposób musimy jedynie podać wartość, której użyjemy do porównania każdego elementu (tzw. klucz). Przy takim podejściu możemy też bardzo łatwo rozwijać nasze komparatory, dodając dodatkowe strategie porównania gdy nasze wartości są równe względem poprzedniego klucza:

```java
SortedSet<Vector2d> xSet = new TreeSet<>(Comparator.comparingInt(Vector2d::x).thenComparing(Vector2d::y));
```
Jeśli porównamy w ten sposób np. wektory A=(2, 4), B=(2, 1) to zapewnimy, że A > B.

# Prawo Demeter

W miejscach użycia klasy `MapBoundary` w Waszych odpowiedziach czasem pojawiał się mniej więcej taki fragment kodu:

```java
public class GrassField {
    
    ... 
        
    @Override
    public Vector2d calculateUpperBound() {
       return new Vector2d(mapBoundary.getSortedElementsX().first().x, mapBoundary.getSortedElementsY().first().y);
    }
   
}
```

W tym fragmencie odwołujemy się do zbioru posortowanych wartości trzymanego w `MapBoundary`. Następnie wyciągamy z niego pierwszy element wiedząc, że będzie to posortowana kolekcja, a następnie jeszcze konkretną składową, wiedząc że to powinno nas w tym momencie interesować. No właśnie, *wiedząc*. Wiedzieć to musi tak naprawdę klasa `GrassField`, która nie tyle korzysta z tego, co oferuje `MapBoundary`, a wręcz steruje jego wewnętrznymi mechanizmami.

Jest to przykład złamania tzw. [prawa Demeter](https://devcezz.pl/2021/12/14/prawo-demeter-jak-uchronic-kod-przed-katastrofa/), które mówi by "nie rozmawiać z nieznajomymi, a jedynie z bliskimi przyjaciółmi". Innymi słowy, nie powinniśmy odwoływać się do atrybutów i metod wywołanych na atrybutach i metodach obiektu, którego używamy. Brzmi skomplikowanie, ale wystarczy pokazać, jak wyglądałby nasz kod gdyby prawo Demeter było tu przestrzegane:

```java
public class GrassField {
    ... 
        
    @Override
    public Vector2d calculateUpperBound() {
       return mapBoundary.getUpperBound();
    }
   
}

public class MapBoundary {
    ...
    
    public Vector2d getUpperBound() {
       return new Vector2d(mapBoundary.getSortedElementsX().first().x, mapBoundary.getSortedElementsY().first().y);
    }
}
```
Teraz szczegóły działania narożników są zamknięte w całości po stronie `MapBoundary`, to jej odpowiedzialność. Dlaczego to wszystko jest takie istotne? Jak zawsze chodzi o czytelność i rozszerzalność kodu. Jeśli np. ktoś nagle stwierdzi, że narożniki będą wyliczane za pomocą innej struktury danych to będziemy musieli jedynie zmienić kod `MapBoundary`. Mieszając odpowiedzialności konieczne byłoby zmienianie zarówno `MapBoundary`, jak i `GrassField` (i potencjalnie innych klas używających tego mechanizmu).



# Wyjątki

Mechanizm wyjątków w Javie jest dość prosty i działa podobnie jak w innych językach - rzucenie wyjątku operatorem `throw` przerywa aktualną metodę i wyjątek leci po stosie wywołań funkcji aż ktoś go powstrzyma, łapiąc go w bloku `catch`. Nie będziemy tu opisywać podstaw obsługi wyjątków, ale skupimy się na jednej istotnej sprawie: czym różnią się wyjątki typu *checked* od tych *unchecked* - i co ważniejsze - kiedy używać jednych, a kiedy drugich.


## Kiedy checked a kiedy unchecked?


Wyjątek typu *unchecked* to taki, który może być rzucony bez żadnego dodatkowego ostrzeżenia. Jeśli chcemy złapać taki wyjątek to musimy _wiedzieć_, że mamy się go spodziewać, bo kompilator nas przed tym nie ostrzeże. Wyjątki *unchecked* to wszystkie, które dziedziczą po `RuntimeException`. Przykładem takiego wyjątku jest `IllegalArgumentException`:

```java
    @Override
    public void place(Animal animal) {
        if (canMoveTo(animal.getLocation())) {
            this.animals.put(animal.getLocation(), animal);
        } else {
            throw new IllegalArgumentException("Animal can't be placed at this position in the map: " + animal.getLocation());
        }
    }
```

Wyjątek *checked* musi być z kolei jasno zasygnalizowany - jeśli wyrzucamy go z metody to musi ona go zadeklarować w sygnaturze `throws`. Dzięki temu każdy, kto wywoła taką metodę będzie zmuszony albo go obsłużyć, albo podać dalej (znów dopisując w sygnaturze `throws`). Ten mechanizm jest kontrolowany na etapie kompilacji programu więc jeśli nie obsłużymy wyjątków, program w ogóle się nie uruchomi. Przykładami takich wyjątków są błędy z rodziny `IOException`. Możemy też oczywiście tworzyć własne:

```java
public List<MoveDirection> parse(List<String> directionsArray) throws UnknownOptionException {
    List<MoveDirection> convertedDirections = new ArrayList<>();
    for (String direction : directionsArray) {

        switch (direction) {
            case "f", "forward" -> convertedDirections.add(MoveDirection.FORWARD);
            case "b", "backward" -> convertedDirections.add(MoveDirection.BACKWARD);
            case "r", "right" -> convertedDirections.add(MoveDirection.RIGHT);
            case "l", "left" -> convertedDirections.add(MoveDirection.LEFT);
            default -> throw new UnknownOptionException(direction + " is not legal move specification");
        }
    }
    return convertedDirections;
}
    
public class UnknownOptionException extends Exception {
    public UnknownOptionException(String message) {
        super(message);
    }
}
```

Pytanie brzmi: po co nam właściwie takie rozrożnienie? Wyjątki mogą opisywać bardzo różne sytuacje, ale zasadniczo możemy podzielić je na dwie grupy: takie, które pojawiają się z winy programisty oraz takie, które są zależne bardziej od warunków, w jakich działa program (a więc pojawiają się "z winy" użytkownika). Ta pierwsza grupa to błędy, które nie powinny się wydarzyć, jeśli program jest napisany perfekcyjnie (co, jak wiadomo, jest ciężkie do uzyskania dla programów, które mają więcej niż 0 linii kodu). Jeśli zapomnimy gdzieś stworzyć obiektu zanim użyjemy zmiennej - dostaniemy `NullPoinerException`. Jeśli do funkcji, która tworzy datę podamy "30.02.2022" to powinniśmy dostać np. `IllegalArgumentException`. Wszystkie te sytuacje to błędy, przed którymi programista powinien się zabezpieczyć, sytuacje których raczej się nie spodziewamy z założenia. 

Co innego jednak, gdy np. wczytujemy z dysku plik. Wówczas może się okazać, że takiego pliku nie ma i poleci `FileNotFoundException`. Nasz program nie jest tu winny, kod napisaliśmy dobry, ale taka sytuacja może się zdarzyć niezależnie od nas. Dlatego w tym przypadku tego typu wyjątek jest *checked* i **musimy** się przed nim zabezpieczyć ilekroć wołamy metodę wczytującą plik. Podobnie zresztą wygląda sytuacja z naszym `OptionParser` - dostaje on dane bezpośrednio od użytkownika więc tutaj powinniśmy się również zabepieczyć przed błędem. Możemy oczywiście użyć `IllegalArgumentException` i pamiętać o tym, by go obsłużyć, ale zdecydowanie fajniej jest, gdy nie musimy pamiętać i kompilator sam nam o tym przypomina - stąd pomysł by tę sytuację obsłużyć własnym wyjątkiem, który jest *checked*.

